In [1]:
from os import getenv
from datetime import datetime, timedelta
from math import degrees
import ephem
from dotenv import load_dotenv

load_dotenv();

In [2]:
BALCONY_LAT = getenv('BALCONY_LAT')
BALCONY_LON = getenv('BALCONY_LON')

# This is not my balcony's elevation, but the elevation of my city above sea level.
# This is due to the way ephem treats an observer's elevation to be the same as the 
# horizon.
ELEVATION = 34.0

In [3]:
class SunObserver(ephem.Observer):
    """Extends an ephem.Observer object to only observe the sun. Adds
    convenient methods to calculate times and positions of sunsets.
    """
    def __init__(self, lat: str | float, lon: str | float, elevation: float) -> None:
        """Initializes the SunObserver object, setting the sun as a
        private property to be accessed from the added methods.

        Args:
            lat: The observer's latitude.
            lon: The observer's longitude.
            elevation: The observer's elevation. ephem treats this as if the horizon is
                also this elevation, so ensure this is the general elevation of the
                surrounding area, rather than if the observer is standing on a high
                object relative to the horizon.
        """
        super().__init__()
        self.lat = str(lat)
        self.lon = str(lon)
        self.elevation = elevation
        self._sun = ephem.Sun()

    def get_sunset_on_date(self, d: datetime) -> tuple[datetime, float]:
        """Finds the sunset time and azimuth (horizon angle) on a given date."""
        self.date = d
        sunset_time = self.next_setting(self._sun)

        # Set observer datetime to the date's sunset time to compute azimuth at sunset
        self.date = sunset_time
        self._sun.compute(self)
        return ephem.localtime(sunset_time), degrees(self._sun.az)
    
    def get_next_date_with_sunset_azimuth_greater_than(
            self, min_azimuth: float, start_date: datetime = datetime.now()
        ) -> datetime:
        """Finds the next date on which the sunset will occur with an azimuth greater
        than the given angle.
        
        Args:
            min_azimuth: The minimum sunset azimuth (horizon angle) in degrees.
            start_date: The date from which to begin the iteration.
        """
        for i in range(365):
            d = start_date + timedelta(days=i)
            sunset_time, sunset_azimuth = self.get_sunset_on_date(d)
            if sunset_azimuth > min_azimuth:
                return sunset_time


In [4]:
sun_observer = SunObserver(BALCONY_LAT, BALCONY_LON, ELEVATION)
next_sunset_time, next_sunset_azimuth = sun_observer.get_sunset_on_date(datetime.now())
print(f'The next sunset will be at {next_sunset_time:%Y-%m-%d %H:%M}')
print(f'The sun will set at {next_sunset_azimuth:.0f}° on the horizon.')

The next sunset will be at 2024-03-24 18:27
The sun will set at 274° on the horizon.


## Calculating the next visible sunset from my balcony

The line of sight from my balcony to the horizon is mostly blocked by a very high wall.
The minimum line of sight is around 310°. I would like to know on which day I can expect
to see the sunset from my balcony.

In [5]:
BALCONY_MIN_AZIMUTH = 310

next_visible_sunset_time = sun_observer.get_next_date_with_sunset_azimuth_greater_than(
    BALCONY_MIN_AZIMUTH
)

print(
    f'The next sunset visible from my balcony will be at '
    f'{next_visible_sunset_time:%Y-%m-%d %H:%M}'
)

The next sunset visible from my balcony will be at 2024-06-02 21:21


## Adding complexity with sunset altitude

In truth, I can't actually see the horizon from my balcony. I look over a cemetery, but
there are houses on the other side of the cemetery that block the direct line of sight
from my balcony to the horizon. Therefore, my "sunset" will actually take place a few 
degrees above the horizon.

I need to adjust my calculations to account for this.

In [6]:
# The degrees of altitude at which the line of sight from my balcony to the sun is 
# blocked by buildings.
BALCONY_MIN_ALTITUDE = 2

In [ ]:
# TODO: wind backwards in time from sunset until the sun is at least at the min 
#     altitude. Use this position to calculate the next day I will get sun on my 
#     balcony.